In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

# import time
# # tmp sleep
# time.sleep(3600)
# print('sleep done')

# load feats
train_x,test_x = [],[]
for feat in glob.glob('../features/*.pkl'):
#     if 'tfidf' in feat or 'lr' in feat or 'mnb' in feat:
#         continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/lr_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/gru_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/other_feat.pkl
(95851, 16) (226998, 16)
file path ../features/lstm_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/simple_rnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/tfidf_feat1.pkl
(95851, 30) (226998, 30)
file path ../features/lstm_muse_adj_bn_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lr_fea

In [2]:
print(train_y[:10])
print(len(train_y))
print(sum(train_y[:,0]))

[[1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
95851
9237


In [3]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3,rnd=42):
    kf = KFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((226998,6))
    all_train_loss_l,all_val_loss_l = 0,0
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        
        # train for each class
        for i in range(6):
            if model_name == 'lr':
                model = LogisticRegression()
            if model_name == 'xgb':
                model = XGBClassifier()
            model.fit(curr_x,curr_y[:,i])
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict_proba(curr_x))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict_proba(hold_out_x))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict_proba(test_x)[:,1]
            test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred

print('done')
            
                
            

done


In [4]:
# xgb_res = simple_ens('xgb',k=3)
# sample_submission = pd.read_csv("../input/sample_submission.csv")
# sample_submission[list_classes] = xgb_res
# sample_submission.to_csv("../results/xgb_all_ens_csv.gz", index=False, compression='gzip')
# print(sample_submission.head())
# print('save done')

In [5]:
xgb_res = simple_ens('xgb',k=5,rnd=233)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_all_ens_csv_5fold.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

0
0.0810515557259 0.0836324179441
1
0.0173885089635 0.021991788446
2
0.04148013889 0.0455166021892
3
0.00477305480321 0.0084870073404
4
0.0548938589845 0.0574532060882
5
0.0156955560689 0.0189943556593
this fold avg train 0.0358804455727 avg val 0.0393458962779
0
0.0797793451183 0.0899113719138
1
0.0173676154001 0.020978979249
2
0.0410332750204 0.0477538067655
3
0.00477191988542 0.00799677321904
4
0.0539565949539 0.0616445342466
5
0.015745849473 0.0183191340533
this fold avg train 0.0354424333085 avg val 0.0411007665745
0
0.0809324585447 0.0858460960491
1
0.0172064814701 0.0222718339047
2
0.0409860865669 0.0468203163061
3
0.00460299641961 0.0089456687365
4
0.0540169876005 0.0619329674586
5
0.0150543743231 0.020342335618
this fold avg train 0.0354665641542 avg val 0.0410265363455
0
0.0804850397498 0.0874965350414
1
0.0170603733575 0.0219480927369
2
0.040745332199 0.0479662073318
3
0.00472708071242 0.00800345427768
4
0.0547238672737 0.0586756869841
5
0.0152192493128 0.0202408208309
this 